In [1]:
import tensorflow as tf
from model import DCGAN
from ops import *
from utils import *
from tensorflow.examples.tutorials.mnist import input_data

#Load the MNIST dataset
mnist = input_data.read_data_sets('MNIST_data/')
def conv_out_size_same(size, stride):
    return int(math.ceil(float(size) / float(stride)))

/home/lemac/miniconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
batch = mnist.train.next_batch(64)[0].reshape([64, 28, 28, 1])

In [ ]:
bb = mnist.train.next_batch(64)

In [ ]:
bb[1]

In [ ]:
run_config = tf.ConfigProto()
with tf.Session(config=run_config) as sess:
    dcgan = DCGAN(
      sess,
      input_width=28,
      input_height=28,
      output_width=28,
      output_height=28,
      batch_size=64,
      sample_num=64,
      y_dim=10,
      z_dim=100,
      dataset_name='mnist',
      input_fname_pattern='*.jpg',
      crop=True,
      checkpoint_dir=None,
      sample_dir=None)

In [ ]:
inputs = dcgan.inputs
inputs

In [ ]:
y = dcgan.y
y

In [ ]:
discriminator = dcgan.D
discriminator

In [ ]:
d_logits = dcgan.D_logits
d_logits

### Understanding the discriminator

In [ ]:
''' self.D, self.D_logits   = self.discriminator(inputs, self.y, reuse=False) '''

def get_d():
    with tf.variable_scope("discriminator") as scope:
        # Inits
        batch_size = 64
        y_dim = 10
        c_dim = 1
        output_height = 28
        output_width = 28
        df_dim=64
        dfc_dim=1024

        # Calcs
        y = tf.placeholder(tf.float32, [batch_size, y_dim], name='y')                         # (64, 10)
        image_dims = [output_height, output_width, c_dim]                                     # (28, 28, 1)
        image = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')     # (64, 28, 28, 1)
        d_bn1 = batch_norm(name='d_bn1')
        d_bn2 = batch_norm(name='d_bn2')

        # Network architecture
        yb = tf.reshape(y, [batch_size, 1, 1, y_dim])                                         # (64, 1, 1, 10)
        x = conv_cond_concat(image, yb)                                                       # (64, 28, 28, 11)

        h0 = lrelu(conv2d(x, c_dim + y_dim, name='d_h0_conv'))                                # (64, 14, 14, 11)
        h0 = conv_cond_concat(h0, yb)                                                         # (64, 14, 14, 21)

        h1 = lrelu(d_bn1(conv2d(h0, df_dim + y_dim, name='d_h1_conv')))                       # (64, 7, 7, 74)
        h1 = tf.reshape(h1, [batch_size, -1])                                                 # (64, 3626)
        h1 = concat([h1, y], 1)                                                               # (64, 3636)

        h2 = lrelu(d_bn2(linear(h1, dfc_dim, 'd_h2_lin')))                                    # (64, 1024)
        h2 = concat([h2, y], 1)                                                               # (64, 1034) 

        h3 = linear(h2, 1, 'd_h3_lin')                                                        # (64, 1) 

        output = tf.nn.sigmoid(h3)                                                            # (64, 1)

        return output, h3

In [ ]:
def get_generator(output_height, output_width, batch_size, y_dim, c_dim, y, z):
    with tf.variable_scope("generator") as scope:
        gfc_dim = 1024
        gf_dim = 64
        g_bn0 = batch_norm(name='g_bn0')
        g_bn1 = batch_norm(name='g_bn1')
        g_bn2 = batch_norm(name='g_bn2')

        s_h, s_w = output_height, output_width
        s_h2, s_h4 = int(s_h/2), int(s_h/4)
        s_w2, s_w4 = int(s_w/2), int(s_w/4)

        # yb = tf.expand_dims(tf.expand_dims(y, 1),2)
        yb = tf.reshape(y, [batch_size, 1, 1, y_dim])
        z = concat([z, y], 1)

        h0 = tf.nn.relu(g_bn0(linear(z, gfc_dim, 'g_h0_lin')))
        h0 = concat([h0, y], 1)

        h1 = tf.nn.relu(g_bn1(
            linear(h0, gf_dim*2*s_h4*s_w4, 'g_h1_lin')))
        h1 = tf.reshape(h1, [batch_size, s_h4, s_w4, gf_dim * 2])

        h1 = conv_cond_concat(h1, yb)

        h2 = tf.nn.relu(g_bn2(deconv2d(h1,
            [batch_size, s_h2, s_w2, gf_dim * 2], name='g_h2')))
        h2 = conv_cond_concat(h2, yb)

        return tf.nn.sigmoid(
            deconv2d(h2, [batch_size, s_h, s_w, c_dim], name='g_h3'))

### Understanding the generator

In [ ]:
# Inits
batch_size = 64
y_dim = 10
c_dim = 1
output_height = 28
output_width = 28
df_dim=64
dfc_dim=1024
z_dim = 100
z = tf.placeholder(tf.float32, [None, z_dim], name='z')
y = tf.placeholder(tf.float32, [batch_size, y_dim], name='y')                         # (64, 10)
d_bn1 = batch_norm(name='d_bn1')
d_bn2 = batch_norm(name='d_bn2')

In [ ]:
# with tf.variable_scope("generator") as scope:
gfc_dim = 1024
gf_dim = 64
g_bn0 = batch_norm(name='g_bn0')
g_bn1 = batch_norm(name='g_bn1')
g_bn2 = batch_norm(name='g_bn2')

s_h, s_w = output_height, output_width
s_h2, s_h4 = int(s_h/2), int(s_h/4)
s_w2, s_w4 = int(s_w/2), int(s_w/4)

# yb = tf.expand_dims(tf.expand_dims(y, 1),2)
yb = tf.reshape(y, [batch_size, 1, 1, y_dim])
z = concat([z, y], 1)

h0 = tf.nn.relu(g_bn0(linear(z, gfc_dim, 'g_h0_lin')))
h0 = concat([h0, y], 1)

h1 = tf.nn.relu(g_bn1(
    linear(h0, gf_dim*2*s_h4*s_w4, 'g_h1_lin')))
h1 = tf.reshape(h1, [batch_size, s_h4, s_w4, gf_dim * 2])

h1 = conv_cond_concat(h1, yb)

h2 = tf.nn.relu(g_bn2(deconv2d(h1,
    [batch_size, s_h2, s_w2, gf_dim * 2], name='g_h2')))
h2 = conv_cond_concat(h2, yb)

output = tf.nn.sigmoid(
    deconv2d(h2, [batch_size, s_h, s_w, c_dim], name='g_h3'))

In [ ]:
output

In [ ]:
''' self.D_, self.D_logits_ = self.discriminator(self.G, self.y, reuse=True) '''

def get_d_g():
    with tf.variable_scope("discriminator") as scope:
        
        scope.reuse_variables()
        
        # Inits
        batch_size = 64
        y_dim = 10
        c_dim = 1
        output_height = 28
        output_width = 28
        df_dim=64
        dfc_dim=1024
        z_dim = 100
        z = tf.placeholder(tf.float32, [None, z_dim], name='z')


        # Calcs
        y = tf.placeholder(tf.float32, [batch_size, y_dim], name='y')                         # (64, 10)
        d_bn1 = batch_norm(name='d_bn1')
        d_bn2 = batch_norm(name='d_bn2')

        G = get_generator(output_height, output_width, batch_size, y_dim, c_dim, y, z)        # (64, 28, 28, 1)

        # Network architecture
        yb = tf.reshape(y, [batch_size, 1, 1, y_dim])                                         # (64, 1, 1, 10)
        x = conv_cond_concat(G, yb)                                                           # (64, 28, 28, 11)
        h0 = lrelu(conv2d(x, c_dim + y_dim, name='d_h0_conv'))                                # (64, 14, 14, 11)
        h0 = conv_cond_concat(h0, yb)                                                         # (64, 14, 14, 21)

        h1 = lrelu(d_bn1(conv2d(h0, df_dim + y_dim, name='d_h1_conv')))                       # (64, 7, 7, 74)
        h1 = tf.reshape(h1, [batch_size, -1])                                                 # (64, 3626)
        h1 = concat([h1, y], 1)                                                               # (64, 3636)

        h2 = lrelu(d_bn2(linear(h1, dfc_dim, 'd_h2_lin')))                                    # (64, 1024)
        h2 = concat([h2, y], 1)                                                               # (64, 1034) 

        h3 = linear(h2, 1, 'd_h3_lin')                                                        # (64, 1) 

        output = tf.nn.sigmoid(h3)                                                            # (64, 1)

        return output, h3

In [ ]:
D, D_logits = get_d()
D_, D_logits_ = get_d_g()

### Training

In [ ]:
def sigmoid_cross_entropy_with_logits(x, y):
    try:
        return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, labels=y)
    except:
        return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, targets=y)


d_loss_real = tf.reduce_mean(
  sigmoid_cross_entropy_with_logits(D_logits, tf.ones_like(D)))
d_loss_fake = tf.reduce_mean(
  sigmoid_cross_entropy_with_logits(D_logits_, tf.zeros_like(D_)))
g_loss = tf.reduce_mean(
  sigmoid_cross_entropy_with_logits(D_logits_, tf.ones_like(D_)))

#### --Training starts

In [ ]:
# Initializations
d_optim = tf.train.AdamOptimizer(config.learning_rate, beta1=config.beta1) \
          .minimize(self.d_loss, var_list=self.d_vars)
g_optim = tf.train.AdamOptimizer(config.learning_rate, beta1=config.beta1) \
          .minimize(self.g_loss, var_list=self.g_vars)
try:
    tf.global_variables_initializer().run()
except:
    tf.initialize_all_variables().run()

self.g_sum = merge_summary([self.z_sum, self.d__sum,
  self.G_sum, self.d_loss_fake_sum, self.g_loss_sum])
self.d_sum = merge_summary(
    [self.z_sum, self.d_sum, self.d_loss_real_sum, self.d_loss_sum])
self.writer = SummaryWriter("./logs", self.sess.graph)

sample_z = np.random.uniform(-1, 1, size=(self.sample_num , self.z_dim))
    
if config.dataset == 'mnist':
    sample_inputs = self.data_X[0:self.sample_num]
    sample_labels = self.data_y[0:self.sample_num]
else:
    sample_files = self.data[0:self.sample_num]
    sample = [
    get_image(sample_file,
            input_height=self.input_height,
            input_width=self.input_width,
            resize_height=self.output_height,
            resize_width=self.output_width,
            crop=self.crop,
            grayscale=self.grayscale) for sample_file in sample_files]
    
if (self.grayscale):
    sample_inputs = np.array(sample).astype(np.float32)[:, :, :, None]
else:
    sample_inputs = np.array(sample).astype(np.float32)
  

In [ ]:
counter = 1
start_time = time.time()
could_load, checkpoint_counter = self.load(self.checkpoint_dir)
if could_load:
    counter = checkpoint_counter
    print(" [*] Load SUCCESS")
else:
    print(" [!] Load failed...")

# Iterate over epochs
for epoch in xrange(config.epoch):
    if config.dataset == 'mnist':
        batch_idxs = min(len(self.data_X), config.train_size) // config.batch_size # get batch indexes
    else:      
        self.data = glob(os.path.join(
          "./data", config.dataset, self.input_fname_pattern))
        batch_idxs = min(len(self.data), config.train_size) // config.batch_size

    for idx in xrange(0, batch_idxs): # iterate over batches
        if config.dataset == 'mnist':
            batch_images = self.data_X[idx*config.batch_size:(idx+1)*config.batch_size]
            batch_labels = self.data_y[idx*config.batch_size:(idx+1)*config.batch_size]
        else:
            batch_files = self.data[idx*config.batch_size:(idx+1)*config.batch_size]
            batch = [
              get_image(batch_file,
                        input_height=self.input_height,
                        input_width=self.input_width,
                        resize_height=self.output_height,
                        resize_width=self.output_width,
                        crop=self.crop,
                        grayscale=self.grayscale) for batch_file in batch_files]
            if self.grayscale:
            batch_images = np.array(batch).astype(np.float32)[:, :, :, None]
            else:
            batch_images = np.array(batch).astype(np.float32)

    batch_z = np.random.uniform(-1, 1, [config.batch_size, self.z_dim]) \
      .astype(np.float32)

    if config.dataset == 'mnist':
        # Update D network
        _, summary_str = self.sess.run([d_optim, self.d_sum],
        feed_dict={ 
          self.inputs: batch_images,
          self.z: batch_z,
          self.y:batch_labels,
        })
        self.writer.add_summary(summary_str, counter)

        # Update G network
        _, summary_str = self.sess.run([g_optim, self.g_sum],
        feed_dict={
          self.z: batch_z, 
          self.y:batch_labels,
        })
        self.writer.add_summary(summary_str, counter)

        # Run g_optim twice to make sure that d_loss does not go to zero (different from paper)
        _, summary_str = self.sess.run([g_optim, self.g_sum],
        feed_dict={ self.z: batch_z, self.y:batch_labels })
        self.writer.add_summary(summary_str, counter)

        errD_fake = self.d_loss_fake.eval({
          self.z: batch_z, 
          self.y:batch_labels
        })
        errD_real = self.d_loss_real.eval({
          self.inputs: batch_images,
          self.y:batch_labels
        })
        errG = self.g_loss.eval({
          self.z: batch_z,
          self.y: batch_labels
        })
    else:
        # Update D network
        _, summary_str = self.sess.run([d_optim, self.d_sum],
        feed_dict={ self.inputs: batch_images, self.z: batch_z })
        self.writer.add_summary(summary_str, counter)

        # Update G network
        _, summary_str = self.sess.run([g_optim, self.g_sum],
        feed_dict={ self.z: batch_z })
        self.writer.add_summary(summary_str, counter)

        # Run g_optim twice to make sure that d_loss does not go to zero (different from paper)
        _, summary_str = self.sess.run([g_optim, self.g_sum],
        feed_dict={ self.z: batch_z })
        self.writer.add_summary(summary_str, counter)

        errD_fake = self.d_loss_fake.eval({ self.z: batch_z })
        errD_real = self.d_loss_real.eval({ self.inputs: batch_images })
        errG = self.g_loss.eval({self.z: batch_z})

    counter += 1
    print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" \
      % (epoch, idx, batch_idxs,
        time.time() - start_time, errD_fake+errD_real, errG))

    if np.mod(counter, 100) == 1:
        if config.dataset == 'mnist':
            samples, d_loss, g_loss = self.sess.run(
              [self.sampler, self.d_loss, self.g_loss],
              feed_dict={
                  self.z: sample_z,
                  self.inputs: sample_inputs,
                  self.y:sample_labels,
              }
            )
            save_images(samples, image_manifold_size(samples.shape[0]),
                  './{}/train_{:02d}_{:04d}.png'.format(config.sample_dir, epoch, idx))
            print("[Sample] d_loss: %.8f, g_loss: %.8f" % (d_loss, g_loss)) 
        else:
        try:
            samples, d_loss, g_loss = self.sess.run(
            [self.sampler, self.d_loss, self.g_loss],
            feed_dict={
                self.z: sample_z,
                self.inputs: sample_inputs,
            },
            )
            save_images(samples, image_manifold_size(samples.shape[0]),
                './{}/train_{:02d}_{:04d}.png'.format(config.sample_dir, epoch, idx))
            print("[Sample] d_loss: %.8f, g_loss: %.8f" % (d_loss, g_loss)) 
        except:
            print("one pic error!...")

    if np.mod(counter, 500) == 2:
        self.save(config.checkpoint_dir, counter)

### CapsNet version

In [ ]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [ ]:
def get_d_caps():
    with tf.variable_scope("discriminator") as scope:
        # Inits
#         batch_size = 64
#         y_dim = 10
#         c_dim = 1
#         output_height = 28
#         output_width = 28
#         df_dim=64
#         dfc_dim=1024
        caps1_n_maps = 32
        caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules
        caps1_n_dims = 8

        # Calcs
#         y = tf.placeholder(tf.float32, [batch_size, y_dim], name='y')                         # (64, 10)
#         image_dims = [output_height, output_width, c_dim]                                     # (28, 28, 1)
#         image = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')     # (64, 28, 28, 1)
#         d_bn1 = batch_norm(name='d_bn1')
#         d_bn2 = batch_norm(name='d_bn2')
        conv1_params = {
        "filters": 256,
        "kernel_size": 9,
        "strides": 1,
        "padding": "valid",
        "activation": tf.nn.relu,
        }

        conv2_params = {
            "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
            "kernel_size": 9,
            "strides": 2,
            "padding": "valid",
            "activation": tf.nn.relu
        }
        
        # Network architecture - Primary Capsules
        conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)                             # (?, 20, 20, 256)
        conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)                         # (?, 6, 6, 256)
        caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],                       # (?, 1152, 8)
                       name="caps1_raw")
        caps1_output = squash(caps1_raw, name="caps1_output")                                 # (?, 1152, 8)
        
        # Network architecture - Digit Capsules
        caps2_n_caps = 10
        caps2_n_dims = 16
        init_sigma = 0.1

        W_init = tf.random_normal(
            shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
            stddev=init_sigma, dtype=tf.float32, name="W_init")
        W = tf.Variable(W_init, name="W")
        batch_size = tf.shape(X)[0]
        W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")                       # (?, 1152, 10, 16, 8)
        caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                               name="caps1_output_expanded")                 # (?, 1152, 10, 8, 1)
        caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                           name="caps1_output_tile")
        caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                                     name="caps1_output_tiled")
        caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled, 
                                    name="caps2_predicted")                                  # (?, 1152, 10, 16, 1)
        
        # Network architecture - Routing
        raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")
        routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")
        weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
        weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                                     name="weighted_sum")
        caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")                                    # (?, 1, 10, 16, 1)
        caps2_output_round_1_tiled = tf.tile(
        caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
            name="caps2_output_round_1_tiled")
        agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")
        raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")
        routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
        weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                                   caps2_predicted,
                                                   name="weighted_predictions_round_2")
        weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                             axis=1, keep_dims=True,
                                             name="weighted_sum_round_2")
        caps2_output_round_2 = squash(weighted_sum_round_2,
                                      axis=-2,
                                      name="caps2_output_round_2")
        caps2_output = caps2_output_round_2                                                   # (?, 1, 10, 16, 1) 

        
#         yb = tf.reshape(y, [batch_size, 1, 1, y_dim])                                         # (64, 1, 1, 10)
#         x = conv_cond_concat(image, yb)                                                       # (64, 28, 28, 11)

#         h0 = lrelu(conv2d(x, c_dim + y_dim, name='d_h0_conv'))                                # (64, 14, 14, 11)
#         h0 = conv_cond_concat(h0, yb)                                                         # (64, 14, 14, 21)

#         h1 = lrelu(d_bn1(conv2d(h0, df_dim + y_dim, name='d_h1_conv')))                       # (64, 7, 7, 74)
#         h1 = tf.reshape(h1, [batch_size, -1])                                                 # (64, 3626)
#         h1 = concat([h1, y], 1)                                                               # (64, 3636)

#         h2 = lrelu(d_bn2(linear(h1, dfc_dim, 'd_h2_lin')))                                    # (64, 1024)
#         h2 = concat([h2, y], 1)                                                               # (64, 1034) 

#         h3 = linear(h2, 1, 'd_h3_lin')                                                        # (64, 1) 

#         output = tf.nn.sigmoid(h3)                                                            # (64, 1)

        return output, h3

In [ ]:
# Getting the data
input_height = 28
input_width = 28
c_dim = 1
batch_size = 64
z_dim = 100

# Caps vers
mnist = input_data.read_data_sets('MNIST_data/')
x_placeholder = tf.placeholder("float", shape=[50, 28,28,1], name='x_placeholder')

# Dcgan vers
image_dims = [input_height, input_width, c_dim]
inputs = tf.placeholder(
    tf.float32, [batch_size] + image_dims, name='real_images')

x_image = inputs

In [ ]:
x_placeholder

In [ ]:
x_image

In [ ]:
x_image.get_shape()

d_w1 = tf.get_variable('d_w1', [9, 9, 1, 256], initializer=tf.truncated_normal_initializer(stddev=0.02))
d_b1 = tf.get_variable('d_b1', [256], initializer=tf.constant_initializer(0))

d1 = tf.nn.conv2d(input=x_image, filter=d_w1, strides=[1, 1, 1, 1], padding='VALID')

d1 = d1 + d_b1

with tf.variable_scope('PrimaryCaps_layer'):
    primaryCaps = CapsConv(num_units=8, with_routing=False)
    caps1 = primaryCaps(d1, num_outputs=256, kernel_size=9, stride=2)
    #assert caps1.get_shape() == [128, 1152, 8, 1]

with tf.variable_scope('DigitCaps_layer'):
    digitCaps = CapsConv(num_units=16, with_routing=True)
    caps2 = digitCaps(caps1, num_outputs=10)


d_w3 = tf.get_variable('d_w3', [16*10, 1024], initializer=tf.truncated_normal_initializer(stddev=0.02))
d_b3 = tf.get_variable('d_b3', [1024], initializer=tf.constant_initializer(0))
d3 = tf.reshape(caps2, [-1, 16*10])
d3 = tf.matmul(d3, d_w3)
d3 = d3 + d_b3
d3 = tf.nn.relu(d3)

d_w4 = tf.get_variable('d_w4', [1024, 1], initializer=tf.truncated_normal_initializer(stddev=0.02))
d_b4 = tf.get_variable('d_b4', [1], initializer=tf.constant_initializer(0))

d4 = tf.matmul(d3, d_w4) + d_b4

return tf.nn.sigmoid(d4), d4

In [ ]:
z = tf.truncated_normal([batch_size, z_dim], mean=0, stddev=1, name='z')

In [ ]:
z

In [ ]:
#first deconv block
g_w1 = tf.get_variable('g_w1', [z_dim,102400 ], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.02))
g_b1 = tf.get_variable('g_b1', [102400], initializer=tf.truncated_normal_initializer(stddev=0.02))
g1 = tf.matmul(z, g_w1) + g_b1
g1 = tf.reshape(g1, [-1, 20, 20, 256])
g1 = tf.contrib.layers.batch_norm(g1, epsilon=1e-5, scope='bn1')
g1 = tf.nn.relu(g1)
print(g1.get_shape())

In [ ]:
#CapsNet Implementation

# Primary Capsules layer, return [batch_size, 1152, 8, 1]
with tf.variable_scope('PrimaryCaps_layerGen'):
    primaryCaps = CapsConv2(num_units=8, with_routing=False)
    caps1Gen = primaryCaps(g1, num_outputs=256, kernel_size=9, stride=2)
    #assert caps1.get_shape() == [128, 1152, 8, 1]

# DigitCaps layer, [batch_size, 10, 16, 1]
with tf.variable_scope('DigitCaps_layerGen'):
    digitCaps = CapsConv2(num_units=16, with_routing=True)
    caps2Gen = digitCaps(caps1Gen, num_outputs=10)


# Decoder structure in Fig. 2
# 1. Do masking, how:
with tf.variable_scope('Masking'):
    # a). calc ||v_c||, then do softmax(||v_c||)
    # [batch_size, 10, 16, 1] => [batch_size, 10, 1, 1]
    v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2Gen),
                                          axis=2, keep_dims=True) + 1e-9)
    softmax_v = tf.nn.softmax(v_length, dim=1)
    assert softmax_v.get_shape() == [50, 10, 1, 1]

    # b). pick out the index of max softmax val of the 10 caps
    # [batch_size, 10, 1, 1] => [batch_size] (index)
    argmax_idx = tf.to_int32(tf.argmax(softmax_v, axis=1))
    assert argmax_idx.get_shape() == [50, 1, 1]
    argmax_idx = tf.reshape(argmax_idx, shape=(50, ))

    # Method 1.
    if not True:
        # c). indexing
        # It's not easy to understand the indexing process with argmax_idx
        # as we are 3-dim animal
        masked_v = []
        for batch_size in range(50):
            v = caps2Gen[batch_size][argmax_idx[batch_size], :]
            masked_v.append(tf.reshape(v, shape=(1, 1, 16, 1)))

        masked_v = tf.concat(masked_v, axis=0)
        assert masked_v.get_shape() == [50, 1, 16, 1]
    # Method 2. masking with true label, default mode
    else:
        # masked_v = tf.matmul(tf.squeeze(caps2), tf.reshape(Y, (-1, 10, 1)), transpose_a=True)
        masked_v = tf.multiply(tf.squeeze(caps2Gen), tf.reshape(np.random.uniform(0.0,9.0,size=10).astype(np.float32), (-1, 10, 1)))
        v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2Gen), axis=2, keep_dims=True) + 1e-9)


# 2. Reconstructe the MNIST images with 3 FC layers
# [batch_size, 1, 16, 1] => [batch_size, 16] => [batch_size, 512]
with tf.variable_scope('Decoder'):
    vector_j = tf.reshape(caps2Gen, shape=(50, -1))
    fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512)
    assert fc1.get_shape() == [50, 512]
    fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024)
    assert fc2.get_shape() == [50, 1024]
    decoded = tf.contrib.layers.fully_connected(fc2, num_outputs=784, activation_fn=tf.sigmoid)

decoded

### Comparing generators

In [40]:
gfc_dim = 1024
gf_dim = 64
batch_size = 64
y_dim = 10
c_dim = 1
output_height = 28
output_width = 28
z_dim = 100
z = tf.placeholder(tf.float32, [None, z_dim], name='z')
g_bn0 = batch_norm(name='g_bn0')
g_bn1 = batch_norm(name='g_bn1')
g_bn2 = batch_norm(name='g_bn2')
g_bn3 = batch_norm(name='g_bn3')

In [3]:
s_h, s_w = output_height, output_width

In [7]:
s_h2, s_w2 = conv_out_size_same(s_h, 2), conv_out_size_same(s_w, 2)

In [11]:
s_h2

14

In [12]:
s_w2

14

In [13]:
s_h4, s_w4 = conv_out_size_same(s_h2, 2), conv_out_size_same(s_w2, 2)

In [15]:
s_h4

7

In [16]:
s_h8, s_w8 = conv_out_size_same(s_h4, 2), conv_out_size_same(s_w4, 2)

In [17]:
s_h8

4

In [18]:
s_h16, s_w16 = conv_out_size_same(s_h8, 2), conv_out_size_same(s_w8, 2)

In [19]:
s_h16

2

In [20]:
# project `z` and reshape
z_, h0_w, h0_b = linear(
    z, gf_dim*8*s_h16*s_w16, 'g_h0_lin', with_w=True)

In [21]:
z_

<tf.Tensor 'g_h0_lin/add:0' shape=(?, 2048) dtype=float32>

In [22]:
h0_b

<tf.Variable 'g_h0_lin/bias:0' shape=(2048,) dtype=float32_ref>

In [23]:
h0_w

<tf.Variable 'g_h0_lin/Matrix:0' shape=(100, 2048) dtype=float32_ref>

In [24]:
h0 = tf.reshape(
    z_, [-1, s_h16, s_w16, gf_dim * 8])

In [25]:
h0

<tf.Tensor 'Reshape:0' shape=(?, 2, 2, 512) dtype=float32>

In [28]:
h0 = tf.nn.relu(g_bn0(h0))

In [29]:
h0

<tf.Tensor 'Relu:0' shape=(?, 2, 2, 512) dtype=float32>

In [30]:
h1, h1_w, h1_b = deconv2d(
    h0, [batch_size, s_h8, s_w8, gf_dim*4], name='g_h1', with_w=True)

In [31]:
h1

<tf.Tensor 'g_h1/Reshape:0' shape=(64, 4, 4, 256) dtype=float32>

In [32]:
h1_w

<tf.Variable 'g_h1/w:0' shape=(5, 5, 256, 512) dtype=float32_ref>

In [33]:
h1_b

<tf.Variable 'g_h1/biases:0' shape=(256,) dtype=float32_ref>

In [34]:
h1 = tf.nn.relu(g_bn1(h1))

In [35]:
h1

<tf.Tensor 'Relu_1:0' shape=(64, 4, 4, 256) dtype=float32>

In [36]:
h2, h2_w, h2_b = deconv2d(
    h1, [batch_size, s_h4, s_w4, gf_dim*2], name='g_h2', with_w=True)

In [37]:
h2 = tf.nn.relu(g_bn2(h2))

In [38]:
h3, h3_w, h3_b = deconv2d(
    h2, [batch_size, s_h2, s_w2, gf_dim*1], name='g_h3', with_w=True)

In [41]:
h3 = tf.nn.relu(g_bn3(h3))

In [42]:
h4, h4_w, h4_b = deconv2d(
    h3, [batch_size, s_h, s_w, c_dim], name='g_h4', with_w=True)

In [43]:
output = tf.nn.tanh(h4)

In [44]:
output

<tf.Tensor 'Tanh:0' shape=(64, 28, 28, 1) dtype=float32>